# Load Dataset

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

dataset=pd.read_excel("C:/Users/10253/Desktop/ML-CW-2/Data/trainDataset.xls")
dataset=dataset[dataset["pCR (outcome)"]!=999]
dataset.head(len(dataset)-1)

,ID,pCR (outcome),RelapseFreeSurvival (outcome),Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,TRG002174,1,144.000000,41.0,0,0.0,0.0,1.0,3,3,...,0.517172,0.375126,3.325332,0.002314,3.880772e+06,473.464852,0.000768,0.182615,0.030508,0.000758
1,TRG002178,0,142.000000,39.0,1,1.0,0.0,0.0,3,3,...,0.444391,0.444391,3.032144,0.005612,2.372010e+06,59.459710,0.004383,0.032012,0.001006,0.003685
2,TRG002204,1,135.000000,31.0,0,0.0,0.0,1.0,2,1,...,0.534549,0.534549,2.485848,0.006752,1.540027e+06,33.935384,0.007584,0.024062,0.000529,0.006447
3,TRG002206,0,12.000000,35.0,0,0.0,0.0,1.0,3,3,...,0.506185,0.506185,2.606255,0.003755,6.936741e+06,46.859265,0.005424,0.013707,0.000178,0.004543
4,TRG002210,0,109.000000,61.0,1,0.0,0.0,0.0,2,1,...,0.462282,0.462282,2.809279,0.006521,1.265399e+06,39.621023,0.006585,0.034148,0.001083,0.005626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,TRG002954,0,49.250000,34.3,0,0.0,0.0,1.0,3,3,...,0.418382,0.418382,2.995603,0.004243,1.005061e+06,156.627179,0.002228,0.136015,0.022148,0.002098
395,TRG002955,1,49.250000,46.1,0,0.0,0.0,1.0,3,3,...,0.439568,0.439568,3.056046,0.001339,1.671271e+07,79.989003,0.003282,0.024716,0.000812,0.003078
396,TRG002958,0,48.500000,53.3,0,0.0,0.0,1.0,2,1,...,0.527779,0.527778,1.500000,0.003728,2.132007e+05,0.996746,0.252582,0.007380,0.000037,0.231059
397,TRG002961,0,47.500000,68.8,1,0.0,0.0,0.0,3,3,...,0.313693,0.313693,3.573557,0.001112,2.008034e+07,204.864200,0.001372,0.054063,0.003697,0.001368


In [2]:
# from sklearn.impute import  SimpleImputer
# imp=SimpleImputer(missing_values=np.nan,strategy='mean')
# names=dataset.columns.drop('ID')
# a=imp.fit_transform(dataset.drop('ID',axis=1))
# imp=SimpleImputer(missing_values=999,strategy='mean')
# a=imp.fit_transform(a)
# df=pd.DataFrame(a,columns=
#                 names)
# df

# Data Preprocessing

### Removing duplicate

In [3]:
dataset.drop_duplicates(inplace=True) #because with 119 features, it is unlikely that exact same data happens in real world, it is useless for training and testing; hence remove it before dataset split

### Split Dataset

In [4]:
dataset['pCR (outcome)'].value_counts()

0    299
1     96
Name: pCR (outcome), dtype: int64

In [5]:
y=dataset['pCR (outcome)']
X=dataset.drop(['pCR (outcome)'],axis=1)


X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=y)

### drop columns that are not applicable

In [6]:
class columnDropperTransformer(): #https://stackoverflow.com/questions/68402691/adding-dropping-column-instance-into-a-pipeline
    def __init__(self,columns):
        self.columns=columns

    def fit(self,X,y=None):
        return self

    def transform(self,X,y=None):
        return X.drop(self.columns,axis=1)

    def fit_transform(self,X,y=None):
        return X.drop(self.columns,axis=1)

dropper = columnDropperTransformer(['ID'])
X_train=dropper.fit_transform(X_train)

X_test=dropper.transform(X_test)

### missing data

In [7]:
dataset.isnull().sum()

ID                               0
pCR (outcome)                    0
RelapseFreeSurvival (outcome)    0
Age                              0
ER                               0
                                ..
original_ngtdm_Busyness          0
original_ngtdm_Coarseness        0
original_ngtdm_Complexity        0
original_ngtdm_Contrast          0
original_ngtdm_Strength          0
Length: 120, dtype: int64

In [8]:
from sklearn.impute import SimpleImputer

names=X_train.columns

imp=SimpleImputer(missing_values=np.nan,strategy='mean')
# X_train[:]=imp.fit_transform(X_train)
# X_test[:]=imp.transform(X_test)
X_train=imp.fit_transform(X_train)
X_test=imp.transform(X_test)

imp.missing_values=np.float64(999)
# X_train[:]=imp.fit_transform(X_train)
# X_test[:]=imp.transform(X_test)
X_train=imp.fit_transform(X_train)
X_test=imp.transform(X_test)

### Removing outliers

In [9]:
from sklearn.neighbors import LocalOutlierFactor

# because use LocalOutlierFactor may lose useful information becuase whenever there is an outlier in any features of the sample, the sample will be detected as outlier sample and all its information will be set to 999
# therefore, use other method

class outlierHandlingWithDev():
    def fit(self,X,y=None):
        self.mean=np.zeros(len(X[0]))
        self.std=np.zeros(len(X[0]))
        for i in range(len(X[0])):
            self.mean[i]=X[:,i].mean()
            self.std[i]=X[:,i].std()

        return self

    def transform(self,X,y=None):
        for i in range(len(X)):
            for j in range(len(X[i])):
                if np.abs(X[i][j]-self.mean[j])>3*self.std[j]:
                    X[i][j]=999
        return X

    def fit_transform(self,X,y=None):
        self.fit(X,y)
        return self.transform(X,y)
    # def fit(self,X,y=None):
    #     self.mean=X.mean(axis=0)
    #     self.std=X.std(axis=0)
    #
    #     return self
    #
    # def transform(self,X,y=None):
    #     for i in X.columns:
    #         for j in range(len(X)):
    #             if abs(X[i][j]-self.mean[i])>(3*self.std[i]):
    #                 X[i][j]=999
    #
    #     return X
    #
    # def fit_transform(self,X,y=None):
    #     self.fit(X,y)
    #     return self.transform(X,y)

detector=outlierHandlingWithDev()
detector.fit(X_train)
detector.transform(X_train)
detector.transform(X_test)

X_train=imp.fit_transform(X_train)
X_test=imp.transform(X_test)

X_train=pd.DataFrame(X_train,columns=names)
X_test=pd.DataFrame(X_test,columns=names)

### Binning

### resample

In [10]:
from imblearn.over_sampling import SMOTE

smote=SMOTE()

X_train,y_train=smote.fit_resample(X_train,y_train)


### MinMaxScaler

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
X_train[:]=scaler.fit_transform(X_train)
X_test[:]=scaler.transform(X_test)
X_train

,RelapseFreeSurvival (outcome),Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,HistologyType,LNStatus,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,0.456897,0.191874,1.000000,1.000000,1.000000,0.000000,1.000000,0.500000,0.0,0.000000,...,0.651945,0.667335,0.323326,0.279340,0.003136,0.007559,0.022807,0.063422,0.005191,0.065253
1,0.775862,0.325517,1.000000,1.000000,1.000000,0.000000,0.500000,0.500000,0.0,1.000000,...,0.530456,0.542978,0.470137,0.677938,0.000718,0.013713,0.012871,0.196799,0.042580,0.039096
2,0.482759,0.687807,0.000000,0.000000,0.000000,1.000000,0.500000,0.000000,0.0,1.000000,...,0.417041,0.426887,0.562781,0.054941,0.081159,0.769329,0.000108,0.719984,0.674811,0.000953
3,0.043103,0.150089,1.000000,0.000000,0.000000,0.000000,1.000000,0.500000,0.0,0.000000,...,0.347732,0.355941,0.570773,0.187794,0.006259,0.056030,0.003152,0.294172,0.118040,0.010028
4,0.715517,0.865464,1.000000,1.000000,0.000000,0.000000,0.500000,0.000000,1.0,1.000000,...,0.558902,0.572096,0.404490,0.041034,0.114394,0.010391,0.016340,0.016220,0.000263,0.055444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473,0.757716,0.308502,1.000000,1.000000,1.000000,0.000000,1.000000,0.953039,0.0,0.906077,...,0.091304,0.093459,0.700720,0.243464,0.004023,0.070994,0.005387,0.403776,0.233790,0.016907
474,0.454282,0.365053,0.000000,0.000000,1.000000,0.000000,0.757719,0.515437,0.0,0.515437,...,0.490071,0.501641,0.494613,0.523800,0.000958,0.045122,0.004306,0.502312,0.313650,0.013572
475,0.440622,0.358889,0.000000,0.000000,0.000000,1.000000,0.500000,0.484302,0.0,0.000000,...,0.619175,0.633792,0.362542,0.439467,0.002409,0.011852,0.016488,0.093852,0.030622,0.049426
476,0.407674,0.422152,0.000000,0.000000,0.406409,0.593591,0.500000,0.296796,0.0,0.000000,...,0.628515,0.643353,0.410029,0.476538,0.002608,0.032977,0.011104,0.162008,0.046136,0.033699


### PCA

In [12]:
def is_related(pair1, pair2):
    if (pair1[0] in pair2 or pair1[1] in pair2) and pair1!=pair2 and not (pair1[0]==pair2[1] and pair1[1]==pair2[0]):
        return True
    return False

def add_relation(corr_fea):
    for i in range(len(corr_fea)):
        for j in range(i,len(corr_fea)):
            if i!=j:
                if is_related(corr_fea[i],corr_fea[j]):
                    fea1=tuple(set(corr_fea[i])-set(corr_fea[j]))
                    fea2=tuple(set(corr_fea[j])-set(corr_fea[i]))
                    fea=fea1+fea2
                    if fea not in corr_fea and rev(fea) not in corr_fea:
                        corr_fea.append((fea1+fea2))

def rev(fea):
    f1=fea[0]
    f2=fea[1]

    return (f2,f1)

corr=X_train.corr()
corr=corr.where(abs(corr)>0.9)
count=0
corr_fea=[]
for i in corr.columns:
    for j in corr.columns:
        if i!=j and abs(corr[i][j])>0.9 and ((i,j) not in corr_fea) and ((j,i) not in corr_fea):
            count+=1
            corr_fea.append((i,j))
            add_relation(corr_fea)

In [13]:
from sklearn.decomposition import PCA

new_num_fea=len(X_train.columns)-count

pca=PCA(n_components=new_num_fea)
X_train=pca.fit_transform(X_train)
X_test=pca.transform(X_test)


### Feature Selection

In [14]:
from sklearn.feature_selection import SelectKBest
selector=SelectKBest(k=10)
X_train=selector.fit_transform(X_train,y_train)
X_test=selector.transform(X_test)
X_train

# from sklearn.feature_selection import S

array([[-1.60251400e+00,  5.26448293e-01, -1.14385945e+00, ...,
         4.50162474e-02, -7.54621442e-02,  3.70921755e-02],
       [-7.75117033e-01,  1.12640966e-01, -1.05736880e+00, ...,
         1.29125042e-01,  5.98241064e-03, -6.19482994e-03],
       [ 2.07406523e+00, -7.59828866e-01,  5.99073837e-01, ...,
         7.54495849e-02, -4.14135527e-02,  1.09935847e-01],
       ...,
       [-1.17600388e+00,  1.49667195e+00, -3.04448087e-01, ...,
        -1.37052809e-01,  5.22002769e-02,  1.08727042e-02],
       [-7.73878639e-01,  9.50926049e-01, -6.19307226e-02, ...,
        -2.57383975e-01,  3.15928124e-02,  8.22453462e-05],
       [ 1.50756239e+00, -3.56418341e-01,  4.06521016e-01, ...,
        -3.57727552e-01,  1.23181565e-01, -2.11792916e-02]])

In [15]:

# c=dataset.corr().abs()
# s=c.unstack()
# so=s.sort_values(kind="quicksort")





# delcol=[]
# c=dataset.corr().shape[0]

# names=dataset.corr().columns
#
# for i in range(c):
#     for j in range(i+1,c):
#         if dataset.corr().iloc[i,j]>0.8 :
#             delcol.append((names[i],names[j]))
# print(len(delcol))
#
# import numpy as np
# np.a
#
# for i in range(c):
#     count = 0
#     for j in range(c):
#         if dataset.corr().iloc[i,j]>0.8:
#             count += 1





# Methods

In [16]:

from sklearn.tree import  DecisionTreeClassifier
from sklearn.metrics import classification_report
clf=DecisionTreeClassifier(max_features=4,max_depth=5,class_weight='balanced')
clf.fit(X_train,y_train)
# clf.predict(X_test)

print(classification_report(y_test,clf.predict(X_test)))
# print(classification_report(y_train,clf.predict(X_train)))

              precision    recall  f1-score   support

           0       0.81      0.72      0.76        60
           1       0.35      0.47      0.40        19

    accuracy                           0.66        79
   macro avg       0.58      0.60      0.58        79
weighted avg       0.70      0.66      0.67        79



In [17]:



from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neural_network import MLPClassifier
clf=MLPClassifier(hidden_layer_sizes=(12,23,40,12),activation="relu",max_iter=200,batch_size=20)
# cross_val_score(clf,X_train,y_train,cv=3)


clf.fit(X_train,y_train)
print(classification_report(y_test,clf.predict(X_test)))
confusion_matrix(y_train,clf.predict(X_train))

              precision    recall  f1-score   support

           0       0.75      0.72      0.74        60
           1       0.23      0.26      0.24        19

    accuracy                           0.61        79
   macro avg       0.49      0.49      0.49        79
weighted avg       0.63      0.61      0.62        79



C:\Users\10253\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


array([[235,   4],
       [  0, 239]], dtype=int64)

In [18]:
from sklearn.ensemble import RandomForestClassifier

forest=RandomForestClassifier(class_weight="balanced")
forest.fit(X_train,y_train)
confusion_matrix(forest.predict(X_test),y_test)

array([[48, 13],
       [12,  6]], dtype=int64)